In [1]:
import sys
import os
from operator import itemgetter
from typing import List, Optional, Dict, Any

from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from utils.model_loader import ModelLoader
from exception.custom_exception import ProjectCustomException
from logger import GLOBAL_LOGGER as loger   
from prompt.prompt_library import PROMPT_REGISTRY
from prompt.prompt_metadata import PromptType
from utils.qdrant_vector_db import QdrantVDB

In [2]:
class ConversationalRAG:
    """
    LCEL-based Conversational RAG with lazy retriever initialization.

    Usage:
        rag = ConversationalRAG(session_id="abc")
        rag.load_retriever_from_faiss(index_path="faiss_index/abc", k=5, index_name="index")
        answer = rag.invoke("What is ...?", chat_history=[])
    """

    def __init__(self, session_id: Optional[str], retriever=None):
        try:
            self.session_id = session_id

            # Load LLM and prompts once
            self.llm = self._load_llm()
            self.contextualize_prompt: ChatPromptTemplate = PROMPT_REGISTRY[
                PromptType.CONTEXTUALIZE_QUESTION.value
            ]
            self.qa_prompt: ChatPromptTemplate = PROMPT_REGISTRY[
                PromptType.CONTEXT_QA.value
            ]

            # Lazy pieces
            self.retriever = retriever
            self.chain = None
            if self.retriever is not None:
                self._build_lcel_chain()

            loger.info("ConversationalRAG initialized", session_id=self.session_id)
        except Exception as e:
            loger.error("Failed to initialize ConversationalRAG", error=str(e))
            raise ProjectCustomException("Initialization error in ConversationalRAG", sys)

In [ ]:
ConversationalRAG

__main__.ConversationalRAG